In [38]:
import numpy as np
from sympy.physics.quantum.cg import CG
from sympy.physics.wigner import wigner_9j
%run SU3_implementations.ipynb

In [39]:
# Function to compute the unitary 9-j 
def unitary_9j(j1,j2,j3,j4,j5,j6,j7,j8,j9):
    try:
        return float(wigner_9j(j1,j2,j3,j4,j5,j6,j7,j8,j9).doit())*np.sqrt((2*j3+1)*(2*j7+1)*(2*j8+1)*(2*j6+1))
    except:
        print("It was not possible to compute the 9-j coefficient")
        return 0

In [40]:
# Function to compute two-shell one-body {a+a} triple-barred matrix elements <...||| {a+a}... |||...> 
#                                                    from Restrepo, et. al, (2024), ArrXiv 2405.04679
# Arguments:
# bra: [[f'-], λ'-, μ'-, [f'+], λ'+, μ'+, ρ', λ', μ', S'-, S'+, S'] 
# ket: [[f-], λ-, μ-, [f+], λ+, μ+, ρ, λ, μ, S-, S+, S] 
# op: [η', η, λ_o, μ_o, S_o] 
# rhob: Multiplicity of the triple-barred matrix element
# ========> NOTE: The ρ labels must start in 0, as ρ = 0,1,2,...

def ts_ob_matrix_element(bra, ket, op, rhob):
    
    # Extract the labels   
    fm, lm, mm, fp, lp, mp, rho, l, m, Sm, Sp, S = ket
    fmq, lmq, mmq, fpq, lpq, mpq, rhoq, lq, mq, Smq, Spq, Sq = bra
    etaq, eta, lo, mo, So = op
    
    # Algebraic coefficient
    coeff = (-1)**(eta+1/2+Sp+Spq+lpq-lp+mpq-mp)*np.sqrt( SU3dim(lp, mp) / SU3dim(lpq, mpq) )*np.sqrt( (2*Sp + 1)/(2*Spq + 1) )
    
    # Compute the unitary 9-j spin coefficient
    coeff_9j = unitary_9j(Sm, 1/2, Smq, Sp, 1/2, Spq, S, So, Sq)
    
    # Compute 9-(λ,μ) coefficient involved
    SU3Coeffs9lammu = SU39lammu(lm, mm, etaq, 0  , lmq, mmq,
                                lp, mp, 0   , eta, lpq, mpq,
                                l , m , lo  , mo , lq , mq )  
        
    # Filter 9-(λ,μ) coefficients by the values of ρ
    SU3Coeffs9lammuFilter = SU3Coeffs9lammu[(SU3Coeffs9lammu["rho13"] == rho) & (SU3Coeffs9lammu["rho1234"] == rhoq) & (SU3Coeffs9lammu["rho1324"] == rhob)]
          
    # Triple barred matrix element of lower shell
    SU3rme_lower = SU3rme([fmq,lmq,mmq],[fm,lm,mm],etaq)
        
    # Triple barred matrix element of upper shell
    SU3rme_upper = SU3rme([fp,lp,mp],[fpq,lpq,mpq],eta)

    # Compute the full expression
    return coeff*coeff_9j*float(SU3Coeffs9lammuFilter["value"])*SU3rme_lower*SU3rme_upper

In [41]:
# Function to compute double barred matrix element of neutron-proton operator 
#                               from Troltenier, et. al, (1995), Nuc. Phys. A 586
# Arguments:
# op_type: "n" or "p"
# bra: [[f'_π-],λ'_π-, μ'_π-, [f'_π+], λ'_π+, μ'_π+, ρ'_π, λ'_π, μ'_π, [f'_ν-], λ'_ν-, μ'_ν-, [f'_ν+], λ'_ν+, μ'_ν+, ρ'_ν, λ'_ν, μ'_ν, ρ', λ', μ', K', L', S'_π-, S'_π+, S'_ν-, S'_ν+, S'_π, S'_ν, S', J'] 
# ket: [[f_π-],λ_π-, μ_π-,[f_π+], λ_π+, μ_π+, ρ_π, λ_π, μ_π, [f_ν-], λ_ν-, μ_ν-, [f_ν+], λ_ν+, μ_ν+, ρ_ν, λ_ν, μ_ν, ρ, λ, μ, K, L, S_π-, S_π+, S_ν-, S_ν+, S_π, S_ν, S, J] 
# op: [η'_π, η_π, η'_ν, η_ν, λ_πo, μ_πo, λ_νo, μ_νo, ρ_o, λ_o, μ_o, K_o, L_o, S_πo, S_νo, S_o, J_o] 
# ========> NOTE: The K labels must start in 1, as K = 1,2,3,...
# ========> NOTE: The ρ labels must start in 0, as ρ = 0,1,2,...

def db_matrix_element(op_type, bra, ket, op):
    
    # Extract the labels
    fpm, lpm, mpm, fpp, lpp, mpp , rhop, lp, mp, fnm, lnm, mnm, fnp, lnp, mnp, rhon, ln, mn, rho, l, m, k, L, Spm, Spp, Snm, Snp, Sp, Sn, S, J = ket
    fpmq, lpmq, mpmq, fppq, lppq, mppq, rhopq, lpq, mpq, fnmq, lnmq, mnmq, fnpq, lnpq, mnpq, rhonq,lnq, mnq, rhoq, lq, mq, kq, Lq, Spmq, Sppq, Snmq, Snpq, Spq, Snq, Sq, Jq = bra
    etapq, etap, etanq, etan, lpo, mpo, lno, mno, rhoo, lo, mo, ko, Lo, Spo, Sno, So, Jo = op

    # Compute the two unitary 9-j coefficients
    coeff_9j_1 = unitary_9j(L, Lo, Lq, S, So, Sq, J, Jo, Jq)
    coeff_9j_2 = unitary_9j(Sp, Spo, Spq, Sn, Sno, Snq, S, So, Sq)
    
    # Compute all Wigner coefficients involved
    SU3Coeffs = SU3WignerCoeff(l, m, L, lo, mo, Lo, lq, mq, Lq) 
    
    # Filter over K, K' and Ko
    SU3CoeffsKFilter = SU3Coeffs[(SU3Coeffs["k1"] == k) & (SU3Coeffs["k2"] == ko) & (SU3Coeffs["k3"] == kq)]
    
    # Compute all 9-(λ,μ) coefficients involved
    SU3Coeffs9lammu = SU39lammu(lp, mp, lpo, mpo, lpq, mpq,
                                ln, mn, lno, mno, lnq, mnq,
                                l , m , lo , mo , lq , mq )
    
    # In case the SU(3) irreps do not couple
    if len(SU3Coeffs9lammu) == 0:
        return 0

    # Filter 9-(λ,μ) coefficients by the values of ρ, ρ', ρ_o
    SU3Coeffs9lammuFilter = SU3Coeffs9lammu[(SU3Coeffs9lammu["rho13"] == rho) & (SU3Coeffs9lammu["rho1234"] == rhoq) & (SU3Coeffs9lammu["rho24"] == rhoo)]

    # Identify the values of the ρ
    rhobs  = [int(i[3:])-1 for i in (list(SU3Coeffs.columns)) if i[:3] == "rho"]
    rhopis = list(SU3Coeffs9lammuFilter['rho12'].unique())
    rhonus = list(SU3Coeffs9lammuFilter['rho34'].unique())
    
    # Summation of eq. 10 in Troltenier, et. al, (1995), Nuc. Phys. A 586
    summation = 0
    for rhob in rhobs:
        for rhopi in rhopis:
            for rhonu in rhonus:
                
                # Extract SU(3) Wigner coefficient
                SU3Coeffrhos  = float(SU3CoeffsKFilter[f"rho{rhob+1}"])
                
                # Extract SU(3) 9-(λ,μ) coefficient
                SU39lammurhos = float(SU3Coeffs9lammuFilter[(SU3Coeffs9lammuFilter["rho12"] == rhopi) & (SU3Coeffs9lammuFilter["rho34"] == rhonu) & (SU3Coeffs9lammuFilter["rho1324"] == rhob)]["value"])
                
                # Compute the proper matrix element depending on the operator type
                if (op_type == "p"):
                    #Compute proton triple-barred matrix element 
                    SU3rme = ts_ob_matrix_element([fpmq, lpmq, mpmq, fppq, lppq, mppq, rhopq, lpq, mpq, Spmq, Sppq, Spq],[fpm, lpm, mpm, fpp, lpp, mpp, rhop, lp, mp, Spm, Spp, Sp],[etapq, etap, lpo, mpo, Spo], rhopi)

                if (op_type == "n"):
                    #Compute neutron triple-barred matrix element 
                    SU3rme = ts_ob_matrix_element([fnmq, lnmq, mnmq, fnpq, lnpq, mnpq, rhonq, lnq, mnq, Snmq, Snpq, Snq],[fnm, lnm, mnm, fnp, lnp, mnp, rhon, ln, mn, Snm, Snp, Sn],[etanq, etan, lno, mno, Sno], rhonu)

                summation += SU3Coeffrhos*SU39lammurhos*SU3rme
    
    return summation

# Examples

In [53]:
db_matrix_element("p", [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,2],34,4,[0],0,0,0,34,4,0,60,8,1,4,0,0,0,0,0,0,0,4], [[2,2,2,1],25,2,[1],6,0,0,31,2,[2,2,2,2],34,4,[0],0,0,0,34,4,0,65,6,1,3,1/2,1/2,0,0,0,0,0,3], [5,6,0,0,3,4,0,0,0,3,4,1,1,0,0,0,1])

-0.1017955838328152

In [54]:
db_matrix_element("n", [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,2],34,4,[0],0,0,0,34,4,0,60,8,1,4,0,0,0,0,0,0,0,4], [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,1],32,2,[1],7,0,0,39,2,0,65,6,1,3,0,0,1/2,1/2,0,0,0,3], [0,0,6,7,0,0,3,4,0,3,4,1,1,0,0,0,1])

-0.08977526609317181

In [58]:
ts_ob_matrix_element([[2,2,2,2],26,4,[0],0,0,0,26,4,0,0,0], [[2,2,2,1],25,2,[1],6,0,0,31,2,1/2,1/2,0], [5,6,3,4,0], 0)

-0.82886246529111

In [59]:
ts_ob_matrix_element([[2,2,2,2],34,4,[0],0,0,0,34,4,0,0,0], [[2,2,2,1],32,2,[1],7,0,0,39,2,1/2,1/2,0], [6,7,3,4,0], 0)

-0.6928789768388224

# Example of B(E1) $3^-\rightarrow2^+$  transition as in Restrepo, et. al, (2024), ArrXiv 2405.04679

In [43]:
pp = []
for i in range(5,-1,-1):
    print(i,i+1)
    pp.append(db_matrix_element("p", [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,2],34,4,[0],0,0,0,34,4,0,60,8,1,2,0,0,0,0,0,0,0,2], [[2,2,2,1],25,2,[1],6,0,0,31,2,[2,2,2,2],34,4,[0],0,0,0,34,4,0,65,6,1,3,1/2,1/2,0,0,0,0,0,3], [5,6,0,0,i,i+1,0,0,0,i,i+1,1,1,0,0,0,1]))

5 6
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
4 5
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
3 4
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
2 3
1 2
0 1


In [44]:
pp

[-0.033513440212452755,
 -0.1494608629931636,
 0.08190408303924243,
 0.1803423683435272,
 0,
 0]

In [60]:
nn = []
for i in range(6,-1,-1):
    print(i,i+1)
    nn.append(db_matrix_element("n", [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,2],34,4,[0],0,0,0,34,4,0,60,8,1,2,0,0,0,0,0,0,0,2], [[2,2,2,2],26,4,[0],0,0,0,26,4,[2,2,2,1],32,2,[1],7,0,0,39,2,0,65,6,1,3,0,0,1/2,1/2,0,0,0,3], [0,0,6,7,0,0,i,i+1,0,i,i+1,1,1,0,0,0,1]))

6 7
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
5 6
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
4 5
 ***** XEWU3 DIMENSION OVERFLOW ==============================================================
3 4
2 3
1 2
0 1


In [61]:
nn

[0.04128890687506161,
 -0.04739516608902348,
 -0.15510285891122647,
 0.0722326113334292,
 0.14386345168053072,
 0,
 0]

In [50]:
ppexp = np.array([-0.5245, 0.2910, 0.7517, -0.6500, -2.4657, 2.4376])
nnexp = np.array([-0.4823, -0.3451, 0.5421, 0.7759, -0.9169, -2.9365, 3.0565])

In [51]:
# Parameters
a =  2.2703772518796668 
b =  -1.175719088119022

In [52]:
(2*2+1)/(2*3+1)*( 2.4644*(a*sum(ppexp*pp) + b*sum(nnexp*nn)) )**2
# The value obtained before is 0.00021518739819449415  e²fm²

0.00021478262812364883

# Example of B(E3) $3^-\rightarrow0^+$ transition